In [31]:
from pyspark.sql import SparkSession, SQLContext

class sparkTest:
    def __init__(self):
        self.sparkSession = SparkSession.builder.appName("shud").config("spark.some.config.option", "some-value").getOrCreate()
        self.sqlContext = SQLContext(self.sparkSession)    

        
        
    def mainProcess(self):
        """Initialize the processing"""
        initList = [("https://h_amazon.com/DealsHomeUrl", "false")]
        df = self.sparkSession.createDataFrame(initList, schema=["url", "parsed"])
        self.sqlContext.registerDataFrameAsTable(df, "UrlList")

        df = self.sqlContext.sql("SELECT url, parsed from UrlList where parsed = 'false'")
        indx = 0
        print(len(df.rdd.collect()))
        for url in df.rdd.collect():
            self.parse(url[0])
            df.show()
            df2 = self.sqlContext.sql("SELECT url, parsed from UrlList where parsed = 'false'")
            df = df.union(df2).dropDuplicates()
            if indx > 10:
                break
            indx +=1
        while len(df.rdd.collect()) > 0:
            print("Tour ", indx)
            df.show()
            for url in df.rdd.collect():
                self.parse(url[0])                
            
            df = self.sqlContext.sql("SELECT url, parsed from UrlList where parsed = 'false'")
            
            if indx > 20:
                break
            indx +=1
            
        
    def parse(self, url):
        # Parse url and add newly found urls
        newUrls = []
        #newUrls.append(("https://x1_amazon.com/x1", "false"))
        #newUrls.append(("https://x2_amazon.com/x2", "false"))
        
        #Set current url as parsed
        newUrls.append((url, "true"))
        
        #Get all urls to synchronize and update
        df = self.sqlContext\
            .sql("SELECT url, parsed from UrlList where url <>'%s'" % url)\
            .union(self.sparkSession.createDataFrame(newUrls))\
            .union(self.sparkSession.createDataFrame([(url, "true")]))\
            .dropDuplicates(['url'])
        
        self.sqlContext.registerDataFrameAsTable(df, "UrlList")
        
        

#Testing
st = sparkTest()
st.mainProcess()


def x():
    df = df.union(spark.createDataFrame([("https://x1_amazon.com/xx", "false")]))
    df = df.union(spark.createDataFrame([("https://x2_amazon.com/xx", "false")]))
    df = df.union(spark.createDataFrame([("https://x1_amazon.com/xx", "false")]))
    #df.show()
    sqlContext.registerDataFrameAsTable(df, "UrlList")
    #tt = "table1" in sqlContext.tableNames()
    #df2 = sqlContext.sql("SELECT url, parsed from UrlList where parsed = 'false'")
    #df2.collect()
    l = ['x','y']
    df = spark.createDataFrame((l, 'false'), schema=["url", "parsed"])
    


1
+--------------------+------+
|                 url|parsed|
+--------------------+------+
|https://h_amazon....| false|
+--------------------+------+

Tour  1
+--------------------+------+
|                 url|parsed|
+--------------------+------+
|https://h_amazon....| false|
+--------------------+------+

